# 반복되는 쿼리문 작성

In [6]:
# database에 여러 테이블의 결과를 사용할 수 있음

# tables 
tables = ['tbl_conn_log',
 'tbl_dns_log',
 'tbl_file_log',
 'tbl_http_log',
 'tbl_notice_log',
 'tbl_smb_files_log',
 'tbl_smb_mapping_log',
 'tbl_ssh_log',
 'tbl_ssl_log']

# 1. 여러 테이블에서 결과 확인하기
* for문을 돌면서 결과를 출력하는 코드

In [ ]:
# '%IoT_Dataset%' 데이터셋이 있는 경우를 확인

sql = """
select count() as cnt
from database.{table}

"""

new_df = pd.DataFrame()
for i in tables:
    df, meta = client.execute(sql.format(table=i), with_column_types=True)
    df = list(df[0])
    df.insert(0, i)
    df = [tuple(df)]
    feats = [meta[idx][0] for idx, m in enumerate(meta)]
    feats.insert(0, 'table name')
    new_df = new_df.append(pd.DataFrame(df, columns=feats),  ignore_index=True)

```markdown
|   | table name         | cnt |
|---|--------------------|-----------------|
| 0 | tbl_conn_log    | 100       |
| 1 | tbl_dce_rpc_log | 0               |
| 2 | tbl_dns_log     | 90           |
| 3 | tbl_file_log    | 8          |
| 4 | tbl_ftp_log     | 0               |
```

# 1. Union 쿼리문 작성

In [5]:
# 날짜별로 데이터셋 끌어오기 

day = '20180515'

# make query

sql = """
with 
    logtime - (60*60*9) as utc_time
select utc_time, logtime, src_ip, dst_ip, '{log}' as log
from database.{table}
where toYYYYMMDD(utc_time) == {day}
"""

union='UNION ALL'.join([sql.format(day = day,
                                   table=t, 
                                   log=t.split('tbl_')[1]) for t in tables])

union_sql = """
select *
from
(
{union}
)
"""

In [4]:
print(union_sql.format(union=union))


select *
from
(

with 
    logtime - (60*60*9) as utc_time
select utc_time, logtime, src_ip, dst_ip, 'conn_log' as log
from database.tbl_conn_log
where toYYYYMMDD(utc_time) == 20180515 and hash like '%IoT_Dataset%'
UNION ALL
with 
    logtime - (60*60*9) as utc_time
select utc_time, logtime, src_ip, dst_ip, 'dns_log' as log
from database.tbl_dns_log
where toYYYYMMDD(utc_time) == 20180515 and hash like '%IoT_Dataset%'
UNION ALL
with 
    logtime - (60*60*9) as utc_time
select utc_time, logtime, src_ip, dst_ip, 'file_log' as log
from database.tbl_file_log
where toYYYYMMDD(utc_time) == 20180515 and hash like '%IoT_Dataset%'
UNION ALL
with 
    logtime - (60*60*9) as utc_time
select utc_time, logtime, src_ip, dst_ip, 'http_log' as log
from database.tbl_http_log
where toYYYYMMDD(utc_time) == 20180515 and hash like '%IoT_Dataset%'
UNION ALL
with 
    logtime - (60*60*9) as utc_time
select utc_time, logtime, src_ip, dst_ip, 'notice_log' as log
from database.tbl_notice_log
where toYYYYMMDD(ut

# 2. 여러개 테이블과 여러개의 Feature 동적 쿼리 구성

In [8]:
for mid in [100, 101, 102, 103, 104, 105]:
    ACTIVE_TABLE = 'database.tbl_{}'.format(str(mid))
    [database, table] = ACTIVE_TABLE.split('.')
    meta = client.execute("""SELECT name FROM system.columns WHERE database = '{}' AND table='{}' """.format(database, table))
    print(meta)
    feats = np.array([name[0] for name in meta])[11:]
    
    print("=============================")
    print("{} 테이블의 동적 쿼리 구성".format(mid))
    sql = """
    select key, max_val
    from(
    select ['{feats}'] as key,
            [{max_val}] as max_val
    from {xai_table}
     where  
        minutes >= '{start_time}' and minutes <= '{end_time}'
        and att_name == '{att_name}'
        and src_ip == '{src_ip}'
    ) array join key, max_val
    order by max_val desc
    """.format(
        feats="','".join(feats),
        max_val = ','.join(['max('+f+')' for f in feats]),
        start_time = "{START_TIME}",
        end_time = "{END_TIME}",
        where_value = "{WHERE_VALUE}",
        xai_table = XAI_TABLE,
        att_name = "{ATT_NAME}",
        src_ip = "{SRC_IP}",
        sample_size = 1000
        
    )


    print(sql)

NameError: name 'client' is not defined